## steps

1) load data
2) resample to 100Hz
3) apply sliding window (on complete timeseries of each trial)
  - 3a) uniform window fct?
  - 3b) gaussian filter
5) determine baseline
6) determine min/max
- as a fraction or as delta?

## 0. import packages

In [2]:
import pandas as pd
from metabolics_slips_trips.loader import *
from metabolics_slips_trips.helper import *
import numpy as np
from metabolics_slips_trips.plots import *
import matplotlib.pyplot as plt

## 1. load data

In [3]:
data ="../../data/" #TODO: change to the path where your data is stored!!
ds = DatasetVo2(data)
index = ds.create_index()
index.head()

,id,trial,magnitude
0,S1,trial1,-0.8
1,S1,trial2,0.6
2,S1,trial3,-0.6
3,S1,trial4,-0.4
4,S1,trial5,0.8


##  iterate over the ds and do the following for each trial: 

In [4]:
t_baseline = 10 #time before each perturbation to be analyzed
t_response = 45 #time after each perturbation to be analyzed

precompute matrix to store data in. The max matrix length will be len(dataset)*10

In [5]:
#for each trial, we want the follwing features: 
columns = ['id', 'n_trial','magnitude', 'n_perturbation', 
           'basline_hr', 'max_hr', 'min_hr', 'std_hr', 
           'baseline_vo2', 'max_vo2', 'min_vo2', 'std_vo2', 
           'baseline_vco2', 'max_vo2', 'min_vo2', 'std_vo2',
           'baseline_ve', 'max_ve', 'min_ve', 'std_ve']

In [6]:
features = np.full((len(ds)*10, len(columns)), np.nan)

In [12]:
def compute_features(data, i, t_baseline, t_response):
    #computes the max and min after each perturbation i
    baseline = data[i-t_baseline:i].mean()
    response = data[i:i+t_response]
    return float(baseline), float((response.max()-baseline)/baseline), float((response.min()-baseline)/baseline), float(response.std())
    #this returns the features as a ration of the baseline!if features should be just the delta, use the following line:
   # return float(baseline), float(response.max()-baseline), float(response.min()-baseline), float(response.std())

In [13]:
for n, trial in enumerate(ds[1:]): 
    # extract HR data, vo2 data, vco2 data and ventilation rate
    data = trial.vo2_data
    hr, vo2, vco2, ve = data.HR, data.VO2, data.VCO2, data.VE

    # resample the data to 100Hz (dt is 1/new frequency)
    hr_resampled, vo2_resampled, vco2_resampled, ve_resampled = resample(hr), resample(vo2), \
                resample(vco2), resample(ve)
    
    # apply a smoothing (e.g. sliding window). The fct sliding window has the default 
    # arguments window_length = 10s and dt = 0.01 --> can be adapted!
    hr_smoothed, vo2_smoothed, vco2_smoothed, ve_smoothed = sliding_window(hr_resampled), \
        sliding_window(vo2_resampled), sliding_window(vco2_resampled), sliding_window(ve_resampled)

    #if no sliding window should be applied: 
  #  hr_smoothed, vo2_smoothed, vco2_smoothed, ve_smoothed = hr_resampled, vo2_resampled, vco2_resampled, ve_resampled

    
    all_data = [hr_smoothed, vo2_smoothed, vco2_smoothed, ve_resampled]
    #load the time of each impulse
    perturbations = trial.hs_after_perturbation_times

    #start to fill the matrix
    
    #iterate over the perturbations: 
    for p, time_p in enumerate(perturbations):
        i = p+n*10
        features[i, 0] = int(trial.index.id[0].strip('S')) #participant ID
        features[i, 1] = int(trial.index.trial[0].strip('trial')) #trial number 
        features[i, 2] = trial.index.magnitude[0]
        features[i, 3] = p
        
        for j, data_type in enumerate(all_data): #iterate over HR, vo2, vco2 and ve
             #ith perturbation in that trial 
            baseline, max_value, min_value, std  =compute_features(data_type, time_p, t_baseline, t_response)
            features[i, 4*(j+1):4*(j+1)+4] = [baseline, max_value, min_value, std]
            
    

*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


/tmp/ipykernel_667463/3094601762.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(baseline), float((response.max()-baseline)/baseline), float((response.min()-baseline)/baseline), float(response.std())


*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


/tmp/ipykernel_667463/3094601762.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(baseline), float((response.max()-baseline)/baseline), float((response.min()-baseline)/baseline), float(response.std())


*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


...

/tmp/ipykernel_667463/3094601762.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(baseline), float((response.max()-baseline)/baseline), float((response.min()-baseline)/baseline), float(response.std())


*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


/tmp/ipykernel_667463/3094601762.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(baseline), float((response.max()-baseline)/baseline), float((response.min()-baseline)/baseline), float(response.std())


*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


/tmp/ipykernel_667463/3094601762.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(baseline), float((response.max()-baseline)/baseline), float((response.min()-baseline)/baseline), float(response.std())


In [14]:
features_df = pd.DataFrame(features, columns = columns)

In [15]:
features_df = features_df.dropna()

In [16]:
features_df.to_csv("../../results/features.csv")